In [1]:
import numpy as np
import pandas as pd

In [2]:
train_fn = '../out/oxford_comma/head_train_nocommas_scored.csv'
valid_fn = '../out/oxford_comma/head_val_nocommas_scored.csv'

In [3]:
train_df = pd.read_csv(train_fn, names=['line', 'prefix_tok_len', 'contains', 'probability', 'argmax'])
valid_df = pd.read_csv(valid_fn, names=['line', 'prefix_tok_len', 'contains', 'probability', 'argmax'])
train_df.head(1)

,line,prefix_tok_len,contains,probability,argmax
0,14,140,True,0.18369,290


In [4]:
#from transformers import AutoModelForCausalLM, AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-j-6B', truncation_side='left')
#tokenizer.decode([11, 290, 393, 13, 286])

', and or. of'

', and or. of'

In [5]:
train_df['argmax_is_comma'] = train_df.argmax == 11
valid_df['argmax_is_comma'] = valid_df.argmax == 11

In [6]:
subset_train_df = train_df[train_df.prefix_tok_len > 50]
subset_valid_df = valid_df[valid_df.prefix_tok_len > 50]
len(train_df), len(valid_df)

(2106, 2173)

# Summary statistics

In [7]:
subset_train_df.argmax.value_counts()[:10]

11      1217
290      618
393       66
13        12
286        6
8          4
284        3
447        3
4696       1
1524       1
Name: argmax, dtype: int64

In [8]:
subset_train_df.contains.value_counts(normalize=True)

False    0.572723
True     0.427277
Name: contains, dtype: float64

In [9]:
subset_valid_df.contains.value_counts(normalize=True)

False    0.576999
True     0.423001
Name: contains, dtype: float64

# Analysis

In [10]:
correctness = subset_train_df.argmax_is_comma == subset_train_df.contains
np.mean(correctness), np.std(correctness)/np.sqrt(len(correctness))

(0.6175138399597383, 0.010902655180914371)

In [11]:
correctness = subset_valid_df.argmax_is_comma == subset_valid_df.contains
np.mean(correctness), np.std(correctness)/np.sqrt(len(correctness))

(0.5839091806515301, 0.010950829017520628)

In [12]:
subset_train_df.groupby('contains').mean()

,line,prefix_tok_len,probability,argmax,argmax_is_comma
contains,,,,,
False,9949.765378,221.685413,0.418233,619.656415,0.495606
True,9978.580683,226.029446,0.592955,150.316843,0.769140


In [13]:
subset_valid_df.groupby('contains').mean()

,line,prefix_tok_len,probability,argmax,argmax_is_comma
contains,,,,,
False,9826.570573,224.183062,0.423823,579.884517,0.535500
True,10126.607935,235.812135,0.580665,205.359393,0.746791


In [14]:
subset_train_df.groupby('contains').sem()

,line,prefix_tok_len,probability,argmax,argmax_is_comma
contains,,,,,
False,170.321466,2.954034,0.006473,105.095388,0.014828
True,202.620606,3.551746,0.007280,42.655525,0.014470
